In [1]:
import numpy as np
import torch
import torch.nn as nn
import pandas as pd
import json
from models.layers import BiLSTM

from data_loading import load_data

%load_ext autoreload
%autoreload 2

# Data Loading

In [2]:
data_path = 'data/atis/voc/'

In [53]:
train_query = pd.read_csv('data/raw_data/ms-cntk-atis/atis.train.query.csv', header=None)

In [54]:
train_query.shape

(4978, 1)

In [45]:
X_train = []

In [46]:
for i in range(len(train_query)):
    X_train.append(train_query[0].iloc[i].split())

In [47]:
X_train

[['178',
  '479',
  '902',
  '851',
  '431',
  '444',
  '266',
  '240',
  '168',
  '210',
  '215',
  '236',
  '482',
  '351',
  '240',
  '27',
  '482',
  '827',
  '606',
  '179'],
 ['178',
  '916',
  '429',
  '228',
  '244',
  '444',
  '682',
  '851',
  '247',
  '654',
  '845',
  '606',
  '179'],
 ['178',
  '916',
  '498',
  '827',
  '234',
  '849',
  '482',
  '739',
  '440',
  '435',
  '827',
  '154',
  '210',
  '428',
  '539',
  '905',
  '179'],
 ['178', '296', '197', '444', '810', '851', '667', '179'],
 ['178',
  '730',
  '870',
  '415',
  '444',
  '682',
  '851',
  '678',
  '886',
  '12',
  '378',
  '179'],
 ['178', '479', '617', '180', '428', '853', '444', '312', '851', '597', '179'],
 ['178',
  '916',
  '513',
  '646',
  '196',
  '498',
  '892',
  '654',
  '180',
  '428',
  '444',
  '304',
  '851',
  '339',
  '179'],
 ['178',
  '770',
  '581',
  '827',
  '429',
  '444',
  '682',
  '851',
  '563',
  '216',
  '654',
  '845',
  '179'],
 ['178', '207', '429', '444', '266', '851', '90

In [32]:
train_query[0].iloc[0]

'178 479 932 545 851 423 180 428 444 294 851 527 897 826 573 180 797 482 789 564 179'

In [40]:
train_slots = pd.read_csv('data/raw_data/ms-cntk-atis/atis.train.slots.csv', header=None)

In [49]:
train_slots.shape

(4978, 1)

In [50]:
train_slots.head()

,0
0,128 128 128 128 128 128 48 128 35 100 128 128 ...
1,128 128 128 128 128 128 48 128 78 128 26 33 128
2,128 128 128 128 45 108 128 48 110 128 128 35 1...
3,128 21 128 128 48 128 78 128
4,128 66 119 128 128 48 128 78 21 38 103 128


In [14]:
train_slots[0].iloc[0]

'128 128 128 128 128 128 48 128 35 100 128 128 128 78 128 14 128 128 12 128'

In [15]:
train_intent = pd.read_csv('data/raw_data/ms-cntk-atis/atis.train.intent.csv', header=None)

In [16]:
train_intent[0].iloc[0]

14

# Data Interpritation

In [17]:
vocab = pd.read_csv('data/raw_data/ms-cntk-atis/atis.dict.vocab.csv', header=None)

In [43]:
vocab

,0
0,'d
1,'hare
2,'ll
3,'m
4,'re
5,'s
6,'t
7,'ve
8,0900
9,1


In [37]:
len(vocab)

943

In [26]:
slots = pd.read_csv('data/raw_data/ms-cntk-atis/atis.dict.slots.csv', header=None)

In [36]:
len(slots)

129

In [28]:
intents = pd.read_csv('data/raw_data/ms-cntk-atis/atis.dict.intent.csv', header=None)

In [35]:
len(intents)

26

# Padding stuff

In [72]:
a = [torch.tensor([1, 2, 3, 4, 5, 6]), torch.tensor([10, 20, 300, 400])]

In [73]:
from torch.nn.utils.rnn import pad_sequence

In [74]:
a[0].shape

torch.Size([6])

In [75]:
a[1].shape

torch.Size([4])

In [76]:
pad_sequence(a, padding_value=0, batch_first=True)

tensor([[  1,   2,   3,   4,   5,   6],
        [ 10,  20, 300, 400,   0,   0]])

[tensor([[1, 2, 3],
         [4, 5, 6]]), tensor([[ 10,  20],
         [300, 400]])]

# Testing LSTM

In [1]:
import random
import torch
from models.layers import BiLSTM
from models.bilstm_model import BiLSTMmodel

%load_ext autoreload
%autoreload 2

In [2]:
a = torch.ones(100, 15, 30)

In [3]:
torch.sum(a, dim=1)

tensor([[15., 15., 15.,  ..., 15., 15., 15.],
        [15., 15., 15.,  ..., 15., 15., 15.],
        [15., 15., 15.,  ..., 15., 15., 15.],
        ...,
        [15., 15., 15.,  ..., 15., 15., 15.],
        [15., 15., 15.,  ..., 15., 15., 15.],
        [15., 15., 15.,  ..., 15., 15., 15.]])

tensor([[15., 15., 15.,  ..., 15., 15., 15.],
        [15., 15., 15.,  ..., 15., 15., 15.],
        [15., 15., 15.,  ..., 15., 15., 15.],
        ...,
        [15., 15., 15.,  ..., 15., 15., 15.],
        [15., 15., 15.,  ..., 15., 15., 15.],
        [15., 15., 15.,  ..., 15., 15., 15.]])

In [4]:
# lstm = BiLSTM(900, 100, 64)
lstm = BiLSTMmodel(900, 100, 129, 29, 64)

In [5]:
batch = 128
seq_len = 800
n_slots = 129
n_intens = 29

In [6]:
input = torch.randint(low=0, high=899, size=(batch, seq_len))
input.shape

torch.Size([128, 800])

torch.Size([128, 800])

In [7]:
len([random.randint(0, seq_len-1) for _ in range(batch)])

128

128

In [8]:
intent, slot = lstm(input, torch.tensor([random.randint(1, seq_len-1) for _ in range(batch)]))

/Users/farshov/anaconda3/lib/python3.7/site-packages/torch/nn/functional.py:1320: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
/Users/farshov/anaconda3/lib/python3.7/site-packages/torch/nn/functional.py:1320: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


KeyboardInterrupt: 

KeyboardInterrupt: 

In [9]:
slot.shape == (batch, n_slots, seq_len)

True

In [10]:
intent.shape == (batch, n_intens)

True

In [11]:
lengths = torch.tensor([random.randint(1, seq_len-1) for _ in range(batch)])
lengths.sort(0, descending=True)

(tensor([789, 785, 781, 778, 768, 766, 738, 719, 709, 701, 680, 677, 673, 653,
         652, 651, 650, 641, 636, 635, 624, 617, 614, 614, 613, 607, 600, 598,
         581, 580, 572, 571, 571, 567, 556, 545, 530, 529, 528, 516, 507, 485,
         484, 480, 477, 475, 471, 467, 460, 457, 457, 456, 440, 438, 435, 429,
         422, 411, 407, 404, 404, 403, 403, 398, 396, 395, 389, 385, 384, 374,
         372, 364, 359, 351, 350, 343, 342, 326, 309, 303, 299, 294, 286, 284,
         279, 267, 265, 263, 245, 245, 244, 236, 229, 228, 227, 214, 211, 202,
         199, 197, 188, 182, 182, 165, 164, 152, 143, 139, 135, 130, 125, 118,
         117, 116, 107, 105,  85,  76,  72,  69,  64,  60,  52,  47,  47,  34,
          21,  12]),
 tensor([ 93,  67,  75,  10,  21,  82, 104,  28, 125,  43,  30, 101,  33,  65,
          35, 113,  63,  99,  37,  83,  46,  19,  86,  47, 112, 103,  16, 102,
          27,  51,  74,  38,  85,  69,   3,  52,  95,  42,  76, 109,  22, 122,
          60,  72,  87,  61,  3

In [12]:
torch.sum(input, dim=0).shape

torch.Size([800])

In [13]:
torch.sum(torch.ones(100, 30, 50), dim=1).shape

torch.Size([100, 50])

# Debugging trainer

In [1]:
from models.train import main


%load_ext autoreload
%autoreload 2

In [2]:
main()

DEVICE: cpu
DEVICE: cpu
epoch: 0
epoch: 0


100%|██████████| 7/7 [00:00<00:00, 10.06it/s]



train_loss: 3.2667537346864357, val_loss: 2.4880686828068326
train: intent_acc=0.8050178531264057
val: intent_acc=0.85225, f1_score=3.168343851852763e-05, sent_acc=0.0
_______________________________
epoch: 1
train_loss: 3.2667537346864357, val_loss: 2.4880686828068326
train: intent_acc=0.8050178531264057
val: intent_acc=0.85225, f1_score=3.168343851852763e-05, sent_acc=0.0
_______________________________
epoch: 1


100%|██████████| 7/7 [00:00<00:00, 10.06it/s]



train_loss: 1.8567794408553686, val_loss: 2.17890511240278
train: intent_acc=0.9299159356725146
val: intent_acc=0.8858928571428571, f1_score=3.480150363407549e-05, sent_acc=0.0
_______________________________
epoch: 2
train_loss: 1.8567794408553686, val_loss: 2.17890511240278
train: intent_acc=0.9299159356725146
val: intent_acc=0.8858928571428571, f1_score=3.480150363407549e-05, sent_acc=0.0
_______________________________
epoch: 2


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/farshov/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3296, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-2-263240bbee7e>", line 1, in <module>
    main()
  File "/Users/farshov/Work/SiriusIntentPredictionSlotFilling/models/train.py", line 98, in main
    slot_pred, intent_pred = model(X, lengths)  # (batch, seq_len, n_slots), (batch, n_intents)
  File "/Users/farshov/anaconda3/lib/python3.7/site-packages/torch/nn/modules/module.py", line 489, in __call__
    result = self.forward(*input, **kwargs)
  File "/Users/farshov/Work/SiriusIntentPredictionSlotFilling/models/bilstm_model.py", line 41, in forward
    slot = self.relu(slot)
  File "/Users/farshov/anaconda3/lib/python3.7/site-packages/torch/nn/modules/module.py", line 489, in __call__
    result = self.forward(*input, **kwargs)
  File "/Users/farshov/anaconda3/lib/python3.7/site-packages/torch/nn/modules/act

KeyboardInterrupt: 

KeyboardInterrupt: 

In [25]:
a = torch.tensor([[1, 2], [1, 20]])

In [26]:
a

tensor([[ 1,  2],
        [ 1, 20]])

In [27]:
import torch.nn.functional as F

In [30]:
F.softmax(a.to(torch.float32), dim=1)

tensor([[2.6894e-01, 7.3106e-01],
        [5.6028e-09, 1.0000e+00]])